In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import pickle as pkl
import tqdm
import models.TextCNN as model

np.random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed_all(3407)
torch.backends.cudnn.deterministic = True  # 保证每次结果一样
torch.backends.cudnn.benchmark = False
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号

In [2]:
config = model.Config('ship_data', 'embedding.npz')

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as f


def conv_and_pool(x, conv):
    x = f.relu(conv(x))
    x = x.squeeze(3)
    x = f.max_pool1d(x, x.size(2)).squeeze(2)
    return x


class Model(nn.Module):
    def __init__(self, config):
        super(Model, self).__init__()
        if config.embedding_pretrained is not None:
            self.embedding_1 = nn.Embedding.from_pretrained(config.embedding_pretrained, freeze=False)
            self.embedding_2 = nn.Embedding.from_pretrained(config.embedding_pretrained, freeze=False)
        else:
            self.embedding_1 = nn.Embedding(config.n_vocab, config.embed, padding_idx=config.n_vocab - 1)
            self.embedding_2 = nn.Embedding(config.n_vocab, config.embed, padding_idx=config.n_vocab - 1)
        self.embedding_2.requires_grad = False  # 第二层不训练
        self.convs = nn.ModuleList(
            [nn.Conv2d(1, config.num_filters, (k, config.embed * 2)) for k in config.filter_sizes])
        self.dropout = nn.Dropout(config.dropout)
        self.fc_layers = nn.Sequential(
            nn.Linear(config.num_filters * len(config.filter_sizes),
                      config.num_filters * len(config.filter_sizes) // 2),
            nn.Linear(config.num_filters * len(config.filter_sizes) // 2,
                      config.num_filters * len(config.filter_sizes) // 4),
            nn.Linear(config.num_filters * len(config.filter_sizes) // 4, config.num_classes))

    def forward(self, x):
        out = torch.cat((self.embedding_1(x), self.embedding_2(x)), dim=2)
        out = out.unsqueeze(1)  # 插入维度 进行卷积运算
        out = torch.cat([conv_and_pool(out, conv) for conv in self.convs], 1)
        out = self.dropout(out)
        out = self.fc_layers(out)
        return out

In [9]:
model = Model(config).to(config.device)
model.embedding_2.requires_grad_(False)
print(model.embedding_2.weight)

Parameter containing:
tensor([[-0.1947,  0.2029, -0.0429,  ..., -0.1167,  0.1597, -0.2722],
        [ 0.0180,  0.4529,  0.1710,  ...,  0.2676, -0.0529, -0.1175],
        [ 0.0116,  0.2388, -0.0969,  ...,  0.0824, -0.0366, -0.0747],
        ...,
        [-0.0376,  0.0355, -0.0811,  ..., -0.1944, -0.0868, -0.0264],
        [ 0.9753,  0.4909,  0.2037,  ...,  0.6229,  0.5855,  0.7365],
        [ 0.7958,  0.9731,  0.9490,  ...,  0.0783,  0.8704,  0.6228]],
       device='cuda:0', requires_grad=True)


In [85]:
model(torch.randint(1, 10, [64, 30]).to(config.device)).shape()

torch.Size([64, 8])


TypeError: 'torch.Size' object is not callable